In [1]:
%%capture
### install packages
## google trends package
!pip install pytrends
## large language model libraries
!pip install accelerate 
!pip install bitsandbytes
!pip install optimum
!pip install auto-gptq
!pip install llama-cpp-python
## image generation libraries
!pip install diffusers
!pip install invisible_watermark transformers accelerate safetensors
# kaggle lib for dataset management
!pip install kaggle

In [2]:
# !CMAKE_ARGS="-DLLAMA_CUBLAS=on" pip install llama-cpp-python does not work :/

In [3]:
## Takes google trends trending terms
#connect google trends api
from pytrends.request import TrendReq


def get_trending_terms(country):
    pytrends = TrendReq(hl='en-US', tz=360)
    trending_searches_df = pytrends.trending_searches(pn=country)
    return list(trending_searches_df[0])


In [4]:
country = "united_kingdom"

In [5]:
google_terms = get_trending_terms(country=country)

In [6]:
google_terms

['Leicester City',
 'Transfer deadline day',
 'Adele',
 'Martin Bashir',
 'Liverpool vs Chelsea',
 'Lenny Henry',
 'South Korea',
 'Arsenal',
 'Premier League',
 'Aston Villa vs Newcastle',
 'Newcastle United',
 'Luton Town vs Brighton',
 'Crystal Palace vs Sheffield United',
 'Fulham vs Everton',
 'Imran Khan',
 'Luton Town',
 'Joanna Love Island',
 'Morocco vs South Africa',
 'Everton',
 'Neuralink']

In [7]:
language_model_name="mistral-7b-instruct-v0.2.Q4_K_M.gguf"

In [8]:
!huggingface-cli download TheBloke/Mistral-7B-Instruct-v0.2-GGUF mistral-7b-instruct-v0.2.Q4_K_M.gguf --local-dir . --local-dir-use-symlinks False

Consider using `hf_transfer` for faster downloads. This solution comes with some limitations. See https://huggingface.co/docs/huggingface_hub/hf_transfer for more details.
downloading https://huggingface.co/TheBloke/Mistral-7B-Instruct-v0.2-GGUF/resolve/main/mistral-7b-instruct-v0.2.Q4_K_M.gguf to /root/.cache/huggingface/hub/tmp52rg098x
mistral-7b-instruct-v0.2.Q4_K_M.gguf: 100%|█| 4.37G/4.37G [00:21<00:00, 199MB/s]
./mistral-7b-instruct-v0.2.Q4_K_M.gguf


In [9]:
from llama_cpp import Llama

llm_path=f"/kaggle/working/{language_model_name}"

model = Llama(
    llm_path, 
    n_gpu_layers=35, 
    embedding=True
)

llama_model_loader: loaded meta data with 24 key-value pairs and 291 tensors from /kaggle/working/mistral-7b-instruct-v0.2.Q4_K_M.gguf (version GGUF V3 (latest))
llama_model_loader: Dumping metadata keys/values. Note: KV overrides do not apply in this output.
llama_model_loader: - kv   0:                       general.architecture str              = llama
llama_model_loader: - kv   1:                               general.name str              = mistralai_mistral-7b-instruct-v0.2
llama_model_loader: - kv   2:                       llama.context_length u32              = 32768
llama_model_loader: - kv   3:                     llama.embedding_length u32              = 4096
llama_model_loader: - kv   4:                          llama.block_count u32              = 32
llama_model_loader: - kv   5:                  llama.feed_forward_length u32              = 14336
llama_model_loader: - kv   6:                 llama.rope.dimension_count u32              = 128
llama_model_loader: - kv   7:  

In [10]:
def generate_prompt_llama(term):
    text = f"""
    <s>[INST] 
    Can you write an artistic prompt about  {term} for an AI image generator? 
    Please think of photorealism, pixel art, steampunk, neopunk, anime, manga, cartoon, cinematography styles, 
    and select one of the styles you think it best matches with {term}. 
    If the {term} is human or animal, make sure the prompt should include anatomically correctness.
    Only give the description.
    [/INST]
    """
    output = model(
        text, # Prompt
        max_tokens=100,  # Generate up to 512 tokens
  #stop=["</s>"],   # Example stop token - not necessarily correct for this specific model! Please check before using.
        echo=True,   # Whether to echo the prompt
        temperature=0.4
)
    return output['choices'][0]['text'].replace("\n", " ").split('[/INST]')[1].lstrip()

In [11]:
prompts = [generate_prompt_llama(trend) for trend in google_terms[:5]]


llama_print_timings:        load time =   28580.69 ms
llama_print_timings:      sample time =      62.99 ms /   100 runs   (    0.63 ms per token,  1587.60 tokens per second)
llama_print_timings: prompt eval time =   28580.45 ms /   125 tokens (  228.64 ms per token,     4.37 tokens per second)
llama_print_timings:        eval time =   33092.17 ms /    99 runs   (  334.26 ms per token,     2.99 tokens per second)
llama_print_timings:       total time =   62104.70 ms /   224 tokens
Llama.generate: prefix-match hit

llama_print_timings:        load time =   28580.69 ms
llama_print_timings:      sample time =      62.97 ms /   100 runs   (    0.63 ms per token,  1588.08 tokens per second)
llama_print_timings: prompt eval time =   23382.28 ms /   103 tokens (  227.01 ms per token,     4.41 tokens per second)
llama_print_timings:        eval time =   33030.37 ms /    99 runs   (  333.64 ms per token,     3.00 tokens per second)
llama_print_timings:       total time =   56845.94 ms /   202 

In [12]:
prompts

['Title: "Leicester City: The Steampunk Champions"       In the heart of England, where the Midlands meet the East, lies the vibrant city of Leicester. Its rich history and resilient spirit have given birth to a football team that defies the odds. Imagine Leicester City as a team of steampunk automatons, each player meticulously crafted from brass, copper, and intricate gears.       The',
 'Title: "The Digital Auction Block: A Steampunk Transfer Deadline Day"       In the heart of a bustling city, where gears and clockworks intertwine with the modern world, lies the grandest spectacle of the sports realm: The Digital Auction Block. The Transfer Deadline Day unfolds in this intricate, steam-powered metropolis, where athletes are bought and sold like precious commodities.       The sun sets on the city',
 'Title: "Adele in the Realms of Neopunk"       In the grungy, vibrant world of Neopunk, a soulful melody resonates through the air. The crowd gathers around a dimly lit stage, anticipat

In [13]:
## Get image generator model

img_model_name = "stabilityai/stable-diffusion-xl-base-1.0"

from diffusers import DiffusionPipeline
import torch

pipe = DiffusionPipeline.from_pretrained(img_model_name, torch_dtype=torch.float32, use_safetensors=True, variant="fp16")
pipe.to("cpu")

The cache for model files in Transformers v4.22.0 has been updated. Migrating your old cache. This is a one-time only operation. You can interrupt this and resume the migration later on by calling `transformers.utils.move_cache()`.


0it [00:00, ?it/s]

model_index.json:   0%|          | 0.00/609 [00:00<?, ?B/s]

2024-01-31 21:01:25.162458: E external/local_xla/xla/stream_executor/cuda/cuda_dnn.cc:9261] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
2024-01-31 21:01:25.162647: E external/local_xla/xla/stream_executor/cuda/cuda_fft.cc:607] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
2024-01-31 21:01:25.332503: E external/local_xla/xla/stream_executor/cuda/cuda_blas.cc:1515] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered


Fetching 19 files:   0%|          | 0/19 [00:00<?, ?it/s]

...

tokenizer_2/vocab.json:   0%|          | 0.00/1.06M [00:00<?, ?B/s]

diffusion_pytorch_model.fp16.safetensors:   0%|          | 0.00/167M [00:00<?, ?B/s]

diffusion_pytorch_model.fp16.safetensors:   0%|          | 0.00/167M [00:00<?, ?B/s]

Loading pipeline components...:   0%|          | 0/7 [00:00<?, ?it/s]

StableDiffusionXLPipeline {
  "_class_name": "StableDiffusionXLPipeline",
  "_diffusers_version": "0.25.1",
  "_name_or_path": "stabilityai/stable-diffusion-xl-base-1.0",
  "feature_extractor": [
    null,
    null
  ],
  "force_zeros_for_empty_prompt": true,
  "image_encoder": [
    null,
    null
  ],
  "scheduler": [
    "diffusers",
    "EulerDiscreteScheduler"
  ],
  "text_encoder": [
    "transformers",
    "CLIPTextModel"
  ],
  "text_encoder_2": [
    "transformers",
    "CLIPTextModelWithProjection"
  ],
  "tokenizer": [
    "transformers",
    "CLIPTokenizer"
  ],
  "tokenizer_2": [
    "transformers",
    "CLIPTokenizer"
  ],
  "unet": [
    "diffusers",
    "UNet2DConditionModel"
  ],
  "vae": [
    "diffusers",
    "AutoencoderKL"
  ]
}

In [14]:
import hashlib

def hash_image(filename):
    """This function returns the MD5 hash of the image file passed into it"""
    # make a hash object
    h = hashlib.md5()

    # open file for reading in binary mode
    with open(filename,'rb') as file:
        # loop till the end of the file
        chunk = 0
        while chunk != b'':
            # read only 1024 bytes at a time
            chunk = file.read(1024)
            h.update(chunk)

    # return the hex representation of digest
    return h.hexdigest()

In [15]:
## Generates images from these prompts
!mkdir /kaggle/working/images

from datetime import datetime
time_id=datetime.now().strftime("%d%m%Y%H%M%S")

resources_metadata = []

for idx, prompt in enumerate(prompts):
    out_path = f'/kaggle/working/images/{time_id}_{idx}.png'
    img = pipe(
        prompt=prompt, 
        num_inference_steps=50, 
        num_images_per_prompt=1, 
        #guidance_scale=0.2, 
        #target_size=(4096, 4096), 
        original_size=(1024, 1024)
              ).images[0]
    img.save(out_path)
    ## Add metadata for the generated image
    resources_metadata.append({
      "path": f"{time_id}_{idx}.png",
      "description": prompt,
      "schema": {
        "fields": [
          {"name": "time_created", "description": datetime.now().strftime("%m/%d/%Y-%H.%M.%S"), "type": "datetime"},
          {"name": "generated_prompt", "description": prompt, "type": "string"},
          {"name": "used_language_model", "description": language_model_name, "type": "string"},
          {"name": "trending_term", "description": google_terms[idx], "type": "string"},
          {"name": "used_image_model", "description": img_model_name, "type": "string"},
          {"name": "country", "description": country, "type": "country"},
          {"name": "upscaled", "description": False, "type": "boolean"},
          {"name": "hash", "description": hash_image(out_path), "type": "string"} 
                ]
            }
    })

Token indices sequence length is longer than the specified maximum sequence length for this model (79 > 77). Running this sequence through the model will result in indexing errors
The following part of your input was truncated because CLIP can only handle sequences up to 77 tokens: ['. the']
Token indices sequence length is longer than the specified maximum sequence length for this model (79 > 77). Running this sequence through the model will result in indexing errors
The following part of your input was truncated because CLIP can only handle sequences up to 77 tokens: ['. the']


  0%|          | 0/50 [00:00<?, ?it/s]

The following part of your input was truncated because CLIP can only handle sequences up to 77 tokens: ['the sun sets on the city']
The following part of your input was truncated because CLIP can only handle sequences up to 77 tokens: ['the sun sets on the city']


  0%|          | 0/50 [00:00<?, ?it/s]

  0%|          | 0/50 [00:00<?, ?it/s]

The following part of your input was truncated because CLIP can only handle sequences up to 77 tokens: ['elements']
The following part of your input was truncated because CLIP can only handle sequences up to 77 tokens: ['elements']


  0%|          | 0/50 [00:00<?, ?it/s]

The following part of your input was truncated because CLIP can only handle sequences up to 77 tokens: ['their uniforms a']
The following part of your input was truncated because CLIP can only handle sequences up to 77 tokens: ['their uniforms a']


  0%|          | 0/50 [00:00<?, ?it/s]

In [16]:
## Save images to your local machine / database

In [17]:
# --- Generating `kaggle.json` File ---
import json
from kaggle_secrets import UserSecretsClient

!mkdir -p /root/.kaggle
!touch /root/.kaggle/kaggle.json

username =  UserSecretsClient().get_secret("KAGGLE_USER")
key =  UserSecretsClient().get_secret("KAGGLE_KEY")
api_token = {"username": username,"key": key}

with open('/root/.kaggle/kaggle.json', 'w') as file:
    json.dump(api_token, file)

!chmod 600 /root/.kaggle/kaggle.json

In [18]:
! cp /kaggle/input/trending-generated-images/* /kaggle/working/images/

In [19]:
! ls /kaggle/working/images 

26012024223648_0.png  30012024234738_0.png
26012024223648_1.png  30012024234738_1.png
26012024223648_2.png  30012024234738_2.png
26012024223648_3.png  30012024234738_3.png
26012024223648_4.png  30012024234738_4.png
26012024235350_0.png  31012024210221_0.png
26012024235350_1.png  31012024210221_1.png
26012024235350_2.png  31012024210221_2.png
26012024235350_3.png  31012024210221_3.png
26012024235350_4.png  31012024210221_4.png
26012024_0.png	      372d483e-02e3-4190-b0f9-9e93a57dc242.png
26012024_1.png	      Cape_Verde.png
26012024_2.png	      Palworld.png
26012024_3.png	      Shoaib_Malik_neopunk.png
26012024_4.png	      Tata_Steel.png
29012024000017_0.png  Troy_Deeney.png
29012024000017_1.png  art_scene_and_space.jpg
29012024000017_2.png  ldm_generated_image.png
29012024000017_3.png  out_image_0.png
29012024000017_4.png  out_image_1.jpg
29012024234537_0.png  save_cat.jpg
29012024234537_1.png  space_layin_4fingers.jpg
29012024234537_2.png  steampunk_tiger.png
29012024234537_3.png  tige

In [20]:
##Download and open your metadata
! kaggle datasets metadata sbgonenc96/trending-generated-images

import json

with open("/kaggle/working/dataset-metadata.json") as ds_fh:
    metadata = json.load(ds_fh)

Downloaded metadata to /kaggle/working/dataset-metadata.json


In [21]:
metadata["data"] += resources_metadata

In [22]:
metadata

{'id': 'sbgonenc96/trending-generated-images',
 'id_no': 4359742,
 'datasetSlugNullable': 'trending-generated-images',
 'ownerUserNullable': 'sbgonenc96',
 'usabilityRatingNullable': 0.3125,
 'titleNullable': 'Trending Generated Images',
 'subtitleNullable': '',
 'descriptionNullable': '',
 'datasetId': 4359742,
 'datasetSlug': 'trending-generated-images',
 'hasDatasetSlug': True,
 'ownerUser': 'sbgonenc96',
 'hasOwnerUser': True,
 'usabilityRating': 0.3125,
 'hasUsabilityRating': True,
 'totalViews': 28,
 'totalVotes': 0,
 'totalDownloads': 1,
 'title': 'Trending Generated Images',
 'hasTitle': True,
 'subtitle': '',
 'hasSubtitle': True,
 'description': '',
 'hasDescription': True,
 'isPrivate': True,
 'keywords': [],
 'licenses': [{'nameNullable': 'CC0-1.0', 'name': 'CC0-1.0', 'hasName': True}],
 'collaborators': [],
 'data': [{'path': '31012024210221_0.png',
   'description': 'Title: "Leicester City: The Steampunk Champions"       In the heart of England, where the Midlands meet th

In [23]:
with open('/kaggle/working/images/dataset-metadata.json', 'w') as file:
    json.dump(metadata, file)

In [24]:
! cp /kaggle/working/images/dataset-metadata.json /kaggle/working/dataset-metadata.json

In [25]:
cat /kaggle/working/dataset-metadata.json

{"id": "sbgonenc96/trending-generated-images", "id_no": 4359742, "datasetSlugNullable": "trending-generated-images", "ownerUserNullable": "sbgonenc96", "usabilityRatingNullable": 0.3125, "titleNullable": "Trending Generated Images", "subtitleNullable": "", "descriptionNullable": "", "datasetId": 4359742, "datasetSlug": "trending-generated-images", "hasDatasetSlug": true, "ownerUser": "sbgonenc96", "hasOwnerUser": true, "usabilityRating": 0.3125, "hasUsabilityRating": true, "totalViews": 28, "totalVotes": 0, "totalDownloads": 1, "title": "Trending Generated Images", "hasTitle": true, "subtitle": "", "hasSubtitle": true, "description": "", "hasDescription": true, "isPrivate": true, "keywords": [], "licenses": [{"nameNullable": "CC0-1.0", "name": "CC0-1.0", "hasName": true}], "collaborators": [], "data": [{"path": "31012024210221_0.png", "description": "Title: \"Leicester City: The Steampunk Champions\"       In the heart of England, where the Midlands meet the East, lies the vibrant city

In [26]:
## Update version of the dataset
!kaggle datasets version -p /kaggle/working/images -m "auto update"

Starting upload for file 372d483e-02e3-4190-b0f9-9e93a57dc242.png
100%|██████████████████████████████████████| 2.77M/2.77M [00:00<00:00, 12.6MB/s]
Upload successful: 372d483e-02e3-4190-b0f9-9e93a57dc242.png (3MB)
Starting upload for file 26012024223648_0.png
100%|██████████████████████████████████████| 1.76M/1.76M [00:00<00:00, 9.53MB/s]
Upload successful: 26012024223648_0.png (2MB)
Starting upload for file art_scene_and_space.jpg
100%|█████████████████████████████████████████| 125k/125k [00:00<00:00, 501kB/s]
Upload successful: art_scene_and_space.jpg (125KB)
Starting upload for file tiger_steampunk.png
100%|██████████████████████████████████████| 1.96M/1.96M [00:00<00:00, 10.9MB/s]
Upload successful: tiger_steampunk.png (2MB)
Starting upload for file 26012024_0.png
100%|██████████████████████████████████████| 1.78M/1.78M [00:00<00:00, 8.63MB/s]
Upload successful: 26012024_0.png (2MB)
Starting upload for file 29012024234537_0.png
100%|██████████████████████████████████████| 1.65M/1.65